Импортируем необходимые библиотеки.

In [1]:
import numpy as np
from sympy import *
import math
from math import copysign as sgn, factorial as fact
from mpmath import besselj as Jn
from sympy.physics.quantum.cg import CG
from scipy.special import gamma, binom, sph_harm as Ynm, gammaincc
from scipy.special import spherical_jn
from scipy.linalg import solve
from scipy.integrate import quad
import matplotlib.pyplot as plt
import subprocess
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Введем вспомогательные функции: двойной факториал, функцию суммирования коэффициентов в представлении молекулярной орбитали для сферических гармоник с одинаковыми $l$ и $m$, нахождение координат вектора по координатам его начала и конца.

In [2]:
def dfact(n):
    if (n % 2 == 1):
        res = 1
    else:
        res = 2
    for i in range(res + 2, n + 1, 2):
        res *= i
    return res

def compare(l, m, b):
    for j in range (0, len(b)):
        if l == b[j][0]:
            if m == b[j][1]:
                return j
    return -1

def transform(a):
    k = 0
    b = []
    for line in a:
        if line[3]!=0:
            if line[0]!=0:
                if len(b) == 0:
                    b.append(line)
                else:
                    k = compare(line[0], line[1], b)
                    if k == -1:
                        b.append(line)
                    else:
                        b[k][3]+=line[3]
    return b

def angle(p_expec,p_center):
    """ expected point, center point """
    d = p_expec - p_center
    r = np.linalg.norm(d)
    phi = math.atan2(d[1], d[0])
    theta = float(acos(d[2]/r))
    if phi < 0:
        phi += 2*pi.evalf()
    return [float(r),float(theta), float(phi)]

$$ \text{Зададим функцию переноса сферической гармоники в новый центр} ~ r_b~\text{:} \\ 
r_a^lY_{lm}(\widehat{r_a})e^{-\alpha r_a^2} = 4\pi e^{-\alpha(r_b^2+R^2))}\sum_{s=0}^{l}{\sum_{l'=0}^{\infty}{\sum_{v=-s}^{s}{\sum_{m'=-l'}^{l'}{(-1)^{l'}}}}}C(s,l-s,l,v,m-v)\sqrt{\frac{4\pi(2l+1)!}{(2s+1)!(2l-2s+1)!}}\cdot \\
\cdot R^{l-s}i_{l'}(2\alpha Rr_b)r_b^sY_{sv}(\widehat{r_b})Y_{l'm'}(\widehat{r_b})Y_{l-s,m-v}(\widehat{R_{ab}})Y_{l'm'}^*(\widehat{R_{ab}}), \\
\text{где:}~ r_a=\sqrt{(x_0-x_a)^2+(y_0-y_a)^2+(z_0-z_a)^2}, \theta_a = arctg(\frac{\sqrt{(x_0-x_a)^2+(y_0-y_a)^2)}}{z_0-z_a}), \varphi_a = arctg(\frac{y_0-y_a}{x_0-x_a}), ~ \mathbf{r_b} \text{- аналогично;} \\
R = \vert R_{ab}\vert = \sqrt{(x_a-x_b)^2+(y_a-y_b)^2+(z_a-z_b)^2}, \theta_R = arctg(\frac{\sqrt{(x_a-x_b)^2+(y_a-y_b)^2)}}{z_a-z_b}), \varphi_R = arctg(\frac{y_a-y_b}{x_a-x_b}) ; \\
i_{l'}(2\alpha Rr_b)=(-i)^{l'}j_{l'}(i\cdot 2\alpha Rr_b), \text{где} j_{l'} \text{- сферическая функция Бесселя}; \\
C(s,l-s,l,v,m-v) = \sqrt{\frac{(2l-2s)!(2s)!(l+m)!(l-m)!}{(2l)!(s+v)!(s-v)!(l-s+m-v)!(l-s-m+v)!}}. $$
Суммирование по l' будет проводиться не до бесконечности, а до определенного числа, определяющегося исходя из требуемой точности.
$$ \text{В случае, когда}~ n_a\neq 0, ~\text{имеет место формула:} \\
r_a^{l+2n_a}Y_{lm}(\widehat{r_a})e^{-\alpha r_a^2}=(-1)^{n_a}\cdot\left( \frac{d}{d\alpha}\right ) ^{n_a}f(r_a)\vert_{n_a=0} $$Необходимо отметить, что сразу же после переноса представляет произведение сферических гармоник $Y(\hat{r_b})$ в виде суммы сферических гармоник с соответствующими коэффициентами. Возвращает массив строк: $[C,L,M,s,l',\alpha,A]$, где $A$ - множитель в аргументе функции бесселя: $A=2\alpha R$.

In [3]:
def translation (orb,coord_new,Lmax):
    res=[]
    l=orb[0]
    m=orb[1]
    X=orb[3]
    Y=orb[4]
    Z=orb[5]
    const=orb[6]
    alpha=orb[7]
    R,theta_R,phi_R = angle(coord_new,np.array([X,Y,Z]))
    for s in range(0,l+1):
        for L in range(0,Lmax+1):
            for v in range(-s,s+1):
                if (l-s+m-v >=0) and (l-s-m+v >=0):
                    for M in range(-L,L+1):
                        c1=4*pi*(1j)**L*exp(-alpha*R*R)*R**(l-s)
                        c2 = sqrt(4*pi*fact(2*l+1)/fact(2*s+1)/fact(2*l-2*s+1))
                        c3=sqrt(fact(2*l-2*s)*fact(2*s)*fact(l+m)*fact(l-m)\
                                /fact(2*l)/fact(s+v)/fact(s-v)/fact(l-s+m-v)/fact(l-s-m+v))
                        sph_harm=Ynm(m-v,l-s,phi_R,theta_R)*np.conj(Ynm(M,L,phi_R,theta_R))                        
                        A=1j*2*alpha*R
                        for L_tot in range(abs(s-L),s+L+1):
                            M_tot=v+M
                            C=sqrt((2*s+1)*(2*L+1)/4/pi/(2*L_tot+1))*CG(s,0,L,0,L_tot,0)*CG(s,v,L,M,L_tot,M_tot)
                            const_tot=const*c1*c2*c3*sph_harm*C.doit()
                            const_tot=const_tot.evalf()
                            if abs(const_tot)>1e-20:
                                res.append([const_tot,L_tot,M_tot,s,L,alpha,A])
    return res                    

$$ \text{Задаем функцию умножения двух сферических гармоник друг на друга:}~ \\ Y_{l_1}^{m_1}(\mathbf{r})\cdot Y_{l_2}^{m_2}(\mathbf{r}) = r^{l_1}Y_{l_1}^{m_1}(\theta,\phi)\cdot r^{l_2}Y_{l_2}^{m_2}(\theta,\phi)
  = \sum_{L, M}
    \sqrt{\frac{(2 l_1 + 1) (2 l_2 + 1)}{4 \pi (2 L + 1)}}
    \langle l_1 \, 0 \, l_2 \, 0 | L \, 0 \rangle
    \langle l_1 \, m_1 \, l_2 \, m_2 | L \, M \rangle
    \cdot r^{l_1+l_2-L}Y_L^M (\mathbf{r}).$$
$$ \text{Учитывая условия, при которых коэффициенты Клебша-Гордана не обращаются в 0, получаем ограничения на L и M:}~\\ L\in [\vert{l_1-l_2}\vert; l_1+l_2] \\ M = m_1+m_2 $$
$$\text{На выходе получаем массив, содержащий значения l и m для всех функций получающегося ряда, а также коэффициенты перед ними.}$$

In [4]:
def multiply(l1, m1, C1, l2, m2, C2, orb, n):
    res = zeros(l1+l2-abs(l1-l2)+1, 5)
    j = 0
    for i in range(abs(l1-l2),l1+l2+1):
        CG0 = CG(l1, 0, l2, 0, i, 0)*CG(l1, m1, l2, m2, i, m1+m2)
        C0 = CG0.doit()
        C00 = sqrt((2*l1+1)*(2*l2+1)/4/pi/(2*i+1))*C0
        C = C1*C2*C00
        res[j,0]=i
        res[j,1]=m1+m2
        res[j,2]=l1+l2-i+n
        res[j,3]=C
        res[j,4]=orb
        j+=1
    return res

Задаем функцию домножения на x, y или z, представленных в виде суммы "непрерывных" гармоник: $$\\ x = \sqrt{\frac{2\pi}{3}}\cdot r\left [ Y_1^{-1}(\theta,\phi)-Y_1^1(\theta,\phi) \right ] \\ y = i\sqrt{\frac{2\pi}{3}}\cdot r\left [ Y_1^{-1}(\theta,\phi)+Y_1^1(\theta,\phi)\right ] \\ z = 2\sqrt{\frac{\pi}{3}}\cdot rY_1^0(\theta,\phi) $$

In [5]:
def OneMultiply(function, res, Orb):
    res1 = zeros(1, 5)
    res2 = zeros(1, 5)
    if function == 'X':
        for lines in res:
            x1 = multiply(lines[0],lines[1],lines[3],1,-1,sqrt(2*pi/3),Orb,lines[2])
            x2 = multiply(lines[0],lines[1],lines[3],1,1,-sqrt(2*pi/3),Orb,lines[2])
            res1 = np.vstack((res1, x1))
            res2 = np.vstack((res2, x2))
        res1 = np.vstack((res1, res2))
    elif function == 'Y':
        for lines in res:
            y1 = multiply(lines[0],lines[1],lines[3],1,-1,1j*sqrt(2*pi/3),Orb,lines[2])
            y2 = multiply(lines[0],lines[1],lines[3],1,1,1j*sqrt(2*pi/3),Orb,lines[2])
            res1 = np.vstack((res1, y1))
            res2 = np.vstack((res2, y2))
        res1 = np.vstack((res1, res2))
    elif function == 'Z':
        for lines in res:
            z1 = multiply(lines[0],lines[1],lines[3],1,0,2*sqrt(pi/3),Orb,lines[2])
            res1 = np.vstack((res1, z1))
    return res1

Создадим массив разложения всех необходимых степенных функций по "непрерывным" сферическим гармоникам. Выводим его в виде: $ [l,m,2n_\alpha,C,Orb]$

In [6]:
c0 = 2*sqrt(pi)
SphericalExpansion = []
MaxDeg=[1,2,3]
for Max in MaxDeg:
    for a in range (0,Max+1):
        for b in range (0,Max-a+1):
            c=Max-a-b
            line1 = [0,0,0,c0,1]
            SphericalExpansion1 = []
            SphericalExpansion2 = []
            SphericalExpansion1.append(line1)
            Orb=2**a*3**b*5**c
            for i in range(0,a):
                SphericalExpansion2 = OneMultiply('X',SphericalExpansion1,Orb)
                SphericalExpansion1 = SphericalExpansion2
            for j in range(0,b):
                SphericalExpansion2 = OneMultiply('Y',SphericalExpansion1,Orb)
                SphericalExpansion1 = SphericalExpansion2
            for k in range(0,c):
                SphericalExpansion2 = OneMultiply('Z',SphericalExpansion1,Orb)
                SphericalExpansion1 = SphericalExpansion2
            #SphericalExpansion2 = transform(SphericalExpansion1)
            SphericalExpansion.append(SphericalExpansion2)

SphericalExpansion1 = SphericalExpansion
SphericalExpansion = []
lineS = [[0,0,0,c0,1]]
SphericalExpansion.append(lineS)
for Function in SphericalExpansion1:
    Function1=transform(Function)
    SphericalExpansion.append(Function1)

Считываем геометрию молекулы.

In [7]:
geom = []
geom1 = []
f = open('geomNH2.txt','r')
for line in f:
    element = line.replace('\n','').split(' ')
    geom1.append(element)
f.close()
geom = [list(filter(None, lst)) for lst in geom1]

Считываем базис атомных орбиталей.

In [8]:
basis1 = []
basis2 = []
f = open('basisNH2.txt','r')
for line in f:
    element = line.replace('\n','').split(' ')
    basis1.append(element)
f.close()
basis2 = [list(filter(None, lst)) for lst in basis1]
basis3 = list(filter(None, basis2))

Считываем коэффициенты для молекулярной орбитали.

In [10]:
orbital = []
orbital1 = []
orbital2 = []
f = open('orbitalNH2.txt','r')
for line in f:
    element = line.replace('\n','').split(' ')
    orbital1.append(element)
f.close()
orbital2 = [list(filter(None, lst)) for lst in orbital1]
orbital = list(filter(None, orbital2))

Молекулярная орбиталь представляется в виде: $\Psi(r) = \sum\limits_i C_i(x-X_i)^{a_i}(y-Y_i)^{b_i}(z-Z_i)^{c_i}\sum\limits_j e^{-\alpha_{ij} (r-R_i)^2}$. Сохраним ее в виде массива: $\left [ C,a,b,c,X,Y,Z,R,\alpha, \text{вид орбитали} \right ] $.

In [11]:
Orbital=[]
atom_count=0
orb_count=0
primitive_check=1
n_line=0
for line in basis3:
    if len(line)==1:
        atom_count+=1
        xyz_N = np.array([float(geom[atom_count][2]),float(geom[atom_count][3]),float(geom[atom_count][4])])
        R=np.sqrt(np.sum(xyz_N**2))
    else:
        if primitive_check != int(line[0]):
            orb_count+=n_line
            primitive_check+=1
        if line[1]=='S':
            n_line=1
            maxDeg=0
        elif line[1]=='P':
            n_line=3
            maxDeg=1
        elif line[1]=='D':
            n_line=6
            maxDeg=2
        elif line[1]=='F':
            n_line=10
            maxDeg=3
        for i in range(0,n_line):
            a=b=c=0
            for symbol in orbital[orb_count+i][3]:
                if symbol=='X':
                    a+=1
                elif symbol=='Y':
                    b+=1
                elif symbol=='Z':
                    c+=1
            alpha=float(line[3])
            const=sqrt(((2*alpha/pi)**(3/2))*(2**(2*(a+b+c)))*(alpha**(a+b+c))/dfact(2*a-1)/dfact(2*b-1)/dfact(2*c-1))
            C = float(line[4])*float(orbital[orb_count+i][4])*const.evalf()#*Norm[primitive_check-1]
            Orb=2**a*3**b*5**c
            line_MO=[C,a,b,c,xyz_N[0],xyz_N[1],xyz_N[2],R,alpha,Orb]
            Orbital.append(line_MO)
            
Orbital=np.array(Orbital)

Рассчитаем значение ВФ МО в пробной точке.

In [12]:
coord_prob=np.array([1.0,1.5,-0.5])
R_prob=np.sqrt(np.sum(coord_prob**2))
MO_prob = np.sum(Orbital[:,0]*(coord_prob[0]-Orbital[:,4])**Orbital[:,1]*(coord_prob[1]-Orbital[:,5])**Orbital[:,2]*\
                 (coord_prob[2]-Orbital[:,6])**Orbital[:,3]*\
                 np.exp((-Orbital[:,8]*np.sum([(coord_prob-Orbital[:,4:7])**2],axis=-1)).astype(float)))
MO_prob

0.119863555780210

Разложим степенные функции в сформированном массиве молекулярной орбитали по "непрерывным" сферическим гармоникам.
Заведем массив вида $[№_{ат. орб.}][l,m,2n,X,Y,Z,C,\alpha,\text{вид орбитали}]$

In [13]:
SolidHarmOrbital = []
for orb in Orbital:
    Const = orb[0]
    a = orb[1]
    b = orb[2]
    c = orb[3]
    X=orb[4]
    Y=orb[5]
    Z=orb[6]
    R = orb[7]
    alpha = orb[8]
    orb = orb[9]
    for Function in SphericalExpansion:
        if int(Function[0][4])==orb:
            for line in Function:
                l=int(line[0])
                m=int(line[1])
                n=int(line[2])
                C=line[3]*Const
                Expand = [l,m,n,X,Y,Z,C.evalf(),alpha,orb]
                SolidHarmOrbital.append(Expand)

Вычислим значение ВФ МО в пробной точке после ее переразложения по "непрерывным" сферическим гармоникам. На данный момент она имеет вид: $$\Psi(r_0) = \sum_i C\vert r-R_i\vert^{l+2n}e^{-\alpha \vert r-R_i\vert^2}Y_{lm}(\theta_{i0}\varphi_{i0})$$

In [14]:
ValueInDot2=0
for line in SolidHarmOrbital:
    l=line[0]
    m=line[1]
    N=line[2]
    X=line[3]
    Y=line[4]
    Z=line[5]
    R0, theta0, phi0 = angle(coord_prob,np.array([X,Y,Z]))
    C=line[6]
    alpha=line[7]
    psi = C*R0**(l+N)*Ynm(m,l,phi0,theta0)*exp(-alpha*R0**2)
    ValueInDot2 += psi
re(ValueInDot2.evalf())

0.119863639112649

Выберем точку, в которую будем переносить молекулярную орбиталь. Выберем максимальное l, до которого будем раскладывать экспоненту.

In [15]:
coord_new=np.array([0.0,0.0,0.0])
Lmax=10

Перенесем орбиталь в новый центр, используя заданную ранее функцию.

In [16]:
MO_trans=[]
for line in SolidHarmOrbital:
    MO_trans.append(translation(line,coord_new,Lmax))
len(MO_trans)

281

Создаем одномерный массив строк без разбиения на примитивы.

In [17]:
MO_trans_tot=[]
for orb in MO_trans:
    for line in orb:
        MO_trans_tot.append(line)

Зададим вспомогательные функции для суммирования коэффициентов в конечной молекулярной орбитали для членов с одинаковыми $L,M,s,l',\alpha,A$.

In [18]:
def CompareTrans(line, res):
    for k in range (0, len(res)):
        if (res[k][1]==line[1]) and (res[k][2]==line[2]) and\
        (res[k][3]==line[3]) and (res[k][4]==line[4]) and (res[k][5]==line[5]) and (res[k][6]==line[6]): 
            return k
    return -1

def ChekTrans(massive):
    res=[]
    for line in massive:
        if len(res) == 0:
            res.append(line)
        else:
            k = CompareTrans(line, res)
            if k == -1:
                res.append(line)
            else:
                res[k][0]+=line[0]
    return res

In [19]:
MO_trans_tot=ChekTrans(MO_trans_tot)
MO_trans=np.array(MO_trans_tot)
MO_trans.shape

(5231, 7)

Вычислим значение перенесенной орбитали в пробной точке.

In [20]:
coord_prob = angle(coord_prob,coord_new)
coord_prob=np.array(coord_prob)
M=np.array(MO_trans[:,2],dtype='int')
L=np.array(MO_trans[:,1],dtype='int')
l=np.array(MO_trans[:,4],dtype='int')
A=np.array(MO_trans[:,6],dtype='complex')

val_trans=np.sum(MO_trans[:,0]*Ynm(M[:],L[:],coord_prob[2],coord_prob[1])\
                 *spherical_jn(l[:],A[:]*coord_prob[0])*coord_prob[0]**MO_trans[:,3]\
                 *np.exp((-MO_trans[:,5]*coord_prob[0]**2).astype(float)))
val_trans.evalf()

0.120058397021943 - 5.619e-18*I

Запишем информацию о перенесенной орбитали в текстовый файл. Все обозначения в файле совпадают с приведенными выше.

In [21]:
InteractiveShell.ast_node_interactivity = "none"

In [22]:
with open("TransMO.txt", "w") as file2:
    file2.write("L M s l' alpha Re(C) Im(C) Im(A)"+'\n')
    for line in MO_trans:
        for index in range (1,5):
            file2.write(str(int(line[index])) + " ")
        file2.write(str(line[5]) + " ")
        file2.write(str(re(line[0])) + " ")
        file2.write(str(im(line[0])) + " ")
        file2.write(str(im(line[6])) + '\n')